# library import

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import statsmodels.api as sm
import seaborn as sns
import random
import os

# 스케일 처리
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import  MaxAbsScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler

# sklearn
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split

from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dropout, Conv1D, Flatten, Dense, MaxPooling1D, Reshape
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model

from tensorflow.keras.utils import to_categorical


In [2]:
import tensorflow as tf
from tensorflow import keras

import IPython

import kerastuner as kt

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_2192\3148714721.py:6: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


In [3]:
class config:
    seed = 42
    device = "cuda:0"    
        
    lr = 1e-3
    epochs = 25
    batch_size = 32
    num_workers = 4
    train_5_folds = True
    
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    tf.random.set_seed(seed)

seed_everything(config.seed)

# 데이터 불러오기 및 전처리

In [4]:
# 데이터 불러오기
B7_TM_re2 = pd.read_csv('../data/B7/B7_TM_5x8.csv')
B7_MG = pd.read_csv('../data/B7/B7_MG.csv')
V3_TM_re2 = pd.read_csv('../data/V3/V3_TM_5x8.csv')
V3_MG = pd.read_csv('../data/V3/V3_MG.csv')

# B7과 V3데이터 합치기
TM = pd.concat([B7_TM_re2 ,V3_TM_re2 ], axis=1)
MG = pd.concat([B7_MG ,V3_MG ], axis=1)

MG[MG == 755.0] = 75.5

print(TM.shape, B7_TM_re2.shape, V3_TM_re2.shape, MG.shape, B7_MG.shape, V3_MG.shape)

(40, 494) (40, 266) (40, 228) (36, 494) (36, 266) (36, 228)


In [5]:
# 스케일 변환
def X_scale(X):
    scaler = RobustScaler()
    X_scaled= scaler.fit_transform(X)
    X_scaled = pd.DataFrame(X_scaled)
    return X_scaled

# 이상치 정리
def outlier(X):
    q1 = X.quantile(0.25)
    q3 = X.quantile(0.75)
    IQR = q3-q1
    condition = (X > (q1 - 1.5*IQR )) & (X < (q3 + 1.5*IQR ))
    X_drop = X[condition]
    X_fill = X_drop.fillna(X_drop.interpolate()) # Nan값을 앞뒤의 값의 동일 간격으로 채우기
    X_fill = X_fill.fillna(method='ffill')
    X_fill = X_fill.fillna(method='bfill')
    return X_fill

def X_array(X):
    N = X.shape[1]
    X_re = np.zeros((N, 40, 1))
    for k in range(N):
        X_re[k, :, 0]  = X.T.iloc[k,:]
    return X_re

def X_array_2D(X):
    N = X.shape[1]
    X_re = np.zeros((N, 5, 8))
    for k in range(N):
        for i in range(5):
            X_re[k, i,: ]  = X.T.iloc[k,i*5:i*5+8]
    return X_re


def Y_array(Y):
    N = Y.shape[1]
    Y_re = np.zeros((N,36))
    for k in range(N):
        Y_re[k, :] = Y.T.iloc[k,:]
    return Y_re

# y 스케일 변환
def Y_scale(Y): 
    scaler = StandardScaler()
    Y_scaled = scaler.fit_transform(Y)
    # Y_scaled = pd.DataFrame(Y_scaled)
    return scaler, Y_scaled


In [6]:
# 스케일 처리 및 이상치 제거
X_scaled = X_scale(TM)
X_outliner = outlier(X_scaled)
X = X_array(X_outliner)
X_2D = X_array_2D(X_outliner)

Y_array = Y_array(MG)
scaler, Y = Y_scale(Y_array)

In [7]:
print('X의 shape: ', X.shape ,', Y의 shape:',Y.shape)

X의 shape:  (494, 40, 1) , Y의 shape: (494, 36)


# 기타 함수 정의

In [8]:
def test_pred_plot(list,y_test, y_pred): 

    x_ax = range(36)

    plt.subplots(constrained_layout=True)
    for i,k in zip(range(1,5),list):
        plt.subplot(2,2,i)
        plt.plot(x_ax, y_test[k], label="test", color='y')
        plt.plot(x_ax, y_pred[k], label="pred", color='c')
        plt.legend()
        plt.title(k)
        plt.ylabel('MG Height')
        plt.xlabel('location')
    plt.show()

In [9]:
def test_pred_mean_plot(y_test, y_pred): 
    x_ax = range(36)
    plt.plot(x_ax, pd.DataFrame(y_test).mean(), label="test_mean", color='r')
    plt.plot(x_ax, pd.DataFrame(y_pred).mean(), label="pred_mean", color='b')
    plt.legend()
    plt.ylabel('MG Height')
    plt.xlabel('location')
    plt.show()

In [10]:
# 스케일 역변환
def scaler_inv(y_test, y_pred):
    y_test_rev = scaler.inverse_transform(y_test)
    df_y_test = pd.DataFrame(y_test_rev.T)
    y_pred_rev = scaler.inverse_transform(y_pred)
    df_y_pred = pd.DataFrame(y_pred_rev.T)
    return y_test_rev, y_pred_rev, df_y_test, df_y_pred

# MLP

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
print('X_train의 shape: ', X_train.shape ,', X_test의 shape:',X_test.shape,
     ', y_train의 shape: ', y_train.shape ,', y_test의 shape:',y_test.shape)

X_train, X_val, y_train, y_val  = train_test_split(X_train, y_train, test_size=0.1, random_state=42)
print('X_train의 shape: ', X_train.shape ,', X_test의 shape:',X_val.shape,
     ', y_train의 shape: ', y_train.shape ,', y_test의 shape:',y_val.shape)

X_train의 shape:  (395, 40, 1) , X_test의 shape: (99, 40, 1) , y_train의 shape:  (395, 36) , y_test의 shape: (99, 36)
X_train의 shape:  (355, 40, 1) , X_test의 shape: (40, 40, 1) , y_train의 shape:  (355, 36) , y_test의 shape: (40, 36)


In [12]:
def model_MLP(hp):
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(40,1)))

    # Tune the number of units in the first Dense layer
    # Choose an optimal value between 32-512
    hp_units = hp.Int('units', min_value = 32, max_value = 512, step = 32)
    model.add(keras.layers.Dense(units = hp.Int('units_1', min_value = 32, max_value = 512, step = 32)
                                 , activation = 'relu'))
    model.add(keras.layers.Dense(units = hp.Int('units_2', min_value = 32, max_value = 512, step = 32)
                                 , activation = 'relu'))
    model.add(keras.layers.Dense(units = hp.Int('units_3', min_value = 32, max_value = 512, step = 32)
                                 , activation = 'relu'))
    '''
    model.add(keras.layers.Dense(units = hp.Int('units_4', min_value = 32, max_value = 512, step = 32)
                                 , activation = 'relu'))
    '''
    model.add(keras.layers.Dense(36, activation=keras.layers.LeakyReLU(alpha=hp.Float('alpha', min_value=0.00, max_value=0.5, step=0.05))))

    # Tune the learning rate for the optimizer 
    model.summary()
    model.compile(loss='mse', optimizer='adam', metrics=['mae'])

    return model


In [13]:
tuner = kt.Hyperband(model_MLP,
                     objective = 'val_loss', 
                     max_epochs = 10,
                     overwrite=True,
                     factor = 3)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 40)                0         
                                                                 
 dense (Dense)               (None, 32)                1312      
                                                                 
 dense_1 (Dense)             (None, 32)                1056      
                                                                 
 dense_2 (Dense)             (None, 32)                1056      
                                                                 
 dense_3 (Dense)             (None, 36)                1188      
                                                                 
Total params: 4,612
Trainable params: 4,612
Non-trainable params: 0
_________________________________________________________________


In [14]:
tuner.search(X_train, y_train, epochs = 100, validation_data = (X_val, y_val), callbacks=[tf.keras.callbacks.EarlyStopping('val_loss', patience=3)])
# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]


Trial 30 Complete [00h 00m 06s]
val_loss: 0.18884706497192383

Best val_loss So Far: 0.18884706497192383
Total elapsed time: 00h 02m 47s
INFO:tensorflow:Oracle triggered exit


In [15]:
print(f"""
units_1 : {best_hps.get('units_1')},
units_2 : {best_hps.get('units_2')},
units_3 : {best_hps.get('units_3')}


alpha : {best_hps.get('alpha')}
""")


units_1 : 416,
units_2 : 416,
units_3 : 480


alpha : 0.30000000000000004



# SVM

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
print('X_train의 shape: ', X_train.shape ,', X_test의 shape:',X_test.shape,
     ', y_train의 shape: ', y_train.shape ,', y_test의 shape:',y_test.shape)

X_train, X_val, y_train, y_val  = train_test_split(X_train, y_train, test_size=0.1, random_state=42)
print('X_train의 shape: ', X_train.shape ,', X_test의 shape:',X_val.shape,
     ', y_train의 shape: ', y_train.shape ,', y_test의 shape:',y_val.shape)

X_train의 shape:  (395, 40, 1) , X_test의 shape: (99, 40, 1) , y_train의 shape:  (395, 36) , y_test의 shape: (99, 36)
X_train의 shape:  (355, 40, 1) , X_test의 shape: (40, 40, 1) , y_train의 shape:  (355, 36) , y_test의 shape: (40, 36)


In [17]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers.experimental import RandomFourierFeatures

In [18]:
def model_SVM(hp):
    n_timesteps = 40
    n_features = 1
    n_outputs = 36
    
    # create model object
    model = keras.Sequential()
    model.add(Reshape((40,), input_shape=(n_timesteps,n_features)))
    model.add(keras.Input(shape=(40,)))
    model.add(RandomFourierFeatures(output_dim=hp.Int('output_dim', min_value=32, max_value=1280, step=16),
                                    scale=hp.Int('scale', min_value=1, max_value=20, step=1),
                                    kernel_initializer=hp.Choice('kernal', values = ['Laplacian','Gaussian'])))
    #model.add(Dense(n_outputs, activation=keras.layers.ELU(alpha=hp.Float('alpha', min_value=0.00, max_value=5, step=0.05)))) # , activation=None
    model.add(keras.layers.Dense(36, activation=keras.layers.LeakyReLU(alpha=hp.Float('alpha', min_value=0.00, max_value=0.5, step=0.05))))
    #compilation of model
    model.compile(loss='mse', optimizer='adam', metrics=['mae'])
    return model

In [19]:
tuner = kt.Hyperband(model_SVM,
                     objective = 'val_loss', 
                     max_epochs = 10,
                     overwrite=True,
                     factor = 3)

In [20]:
tuner.search(X_train, y_train, epochs = 100, batch_size=24,  validation_data = (X_val, y_val), callbacks=[tf.keras.callbacks.EarlyStopping('val_loss', patience=20)])
# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]


Trial 30 Complete [00h 00m 02s]
val_loss: 0.6154374480247498

Best val_loss So Far: 0.3391115665435791
Total elapsed time: 00h 01m 46s
INFO:tensorflow:Oracle triggered exit


In [21]:
print(f"""
output_dim : {best_hps.get('output_dim')}
scale : {best_hps.get('scale')}
kernal : {best_hps.get('kernal')}

alpha : {best_hps.get('alpha')}
""")


output_dim : 1280
scale : 20
kernal : Laplacian

alpha : 0.35000000000000003



# 1D-CNN

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
print('X_train의 shape: ', X_train.shape ,', X_test의 shape:',X_test.shape,
     ', y_train의 shape: ', y_train.shape ,', y_test의 shape:',y_test.shape)

X_train, X_val, y_train, y_val  = train_test_split(X_train, y_train, test_size=0.1, random_state=42)
print('X_train의 shape: ', X_train.shape ,', X_test의 shape:',X_val.shape,
     ', y_train의 shape: ', y_train.shape ,', y_test의 shape:',y_val.shape)

X_train의 shape:  (395, 40, 1) , X_test의 shape: (99, 40, 1) , y_train의 shape:  (395, 36) , y_test의 shape: (99, 36)
X_train의 shape:  (355, 40, 1) , X_test의 shape: (40, 40, 1) , y_train의 shape:  (355, 36) , y_test의 shape: (40, 36)


In [23]:
from tensorflow.keras.layers import Embedding, Dropout, Flatten, Dense, Conv2D, MaxPooling2D, BatchNormalization, Activation

def model_1D_CNN(hp):
    # create model object
    model = keras.Sequential()
    
    input_shape =(40,1)

    model = Sequential()
    model.add(Conv1D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [2,3,4]),
        # activation='relu',
        input_shape = input_shape, padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv1D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [2,3,4]),
        # activation='relu',
        padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    '''
    model.add(Conv1D(
        filters=hp.Int('conv_3_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_3_kernel', values = [2,3,4]),
        # activation='relu',
        padding='same'))
    model.add(BatchNormalization())

    model.add(MaxPooling1D(pool_size=hp.Int('MaxPooling2D', min_value=1, max_value=5, step=1)
                          ))
    '''
    model.add(Activation('relu'))
    
    model.add(Flatten())

    model.add(keras.layers.Dense(
        units=hp.Int('dense_1', min_value=32, max_value=128, step=16),
        activation='relu'
    ))
    model.add(Dropout(hp.Float('Dropout', min_value=0.00, max_value=0.5, step=0.05)))    
        

    model.add(keras.layers.Dense(
        units=hp.Int('dense_2', min_value=32, max_value=128, step=16),
        activation='relu'
    ))
    model.add(Dense(36,
                     activation=keras.layers.LeakyReLU(alpha=hp.Float('alpha', min_value=0.00, max_value=0.5, step=0.05))
                    ))
    
    #compilation of modelv
    model.compile(loss='mse', optimizer='adam', metrics=['mae'])
    return model

In [24]:
tuner = kt.Hyperband(model_1D_CNN,
                     objective = 'val_loss', 
                     max_epochs = 10,
                     overwrite=True,
                     factor = 5)

In [25]:
tuner.search(X_train, y_train, epochs = 100, batch_size=12,  validation_data = (X_val, y_val), callbacks=[tf.keras.callbacks.EarlyStopping('val_loss', patience=20)])
# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]


Trial 13 Complete [00h 00m 04s]
val_loss: 0.3488042652606964

Best val_loss So Far: 0.34005075693130493
Total elapsed time: 00h 00m 41s
INFO:tensorflow:Oracle triggered exit


In [26]:
print(f"""
conv_1 : {best_hps.get('conv_1_filter')}
conv_1_kernel : {best_hps.get('conv_1_kernel')}

dense_1 : {best_hps.get('dense_1')}
Dropout : {best_hps.get('Dropout')}
dense_2 : {best_hps.get('dense_2')}
alpha : {best_hps.get('alpha')}
""")


conv_1 : 80
conv_1_kernel : 2

dense_1 : 80
Dropout : 0.15000000000000002
dense_2 : 64
alpha : 0.25



In [27]:
def model_1D_CNN(hp):
    # create model object
    model = keras.Sequential()
    
    # adding first convolutional layer 
    model.add(keras.layers.Conv1D(
        filters= hp.Int('conv_1', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(40,1)
    ))
    # adding flatten layer    
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(
        units=hp.Int('dense_1', min_value=32, max_value=128, step=16),
        activation= 'relu'
    ))
    # adding dropout layer 
    model.add(Dropout(hp.Float('Dropout', min_value=0.00, max_value=0.5, step=0.05)))
    
    # adding dense layer    
    model.add(keras.layers.Dense(
        units=hp.Int('dense_2', min_value=32, max_value=128, step=16),
        activation= 'relu'
    ))
    # output layer    
    model.add(Dense(36, keras.layers.LeakyReLU(alpha=hp.Float('alpha', min_value=0.00, max_value=0.5, step=0.05))))
    
    #compilation of model
    model.compile(loss='mse', optimizer='adam', metrics=['mae'])
    return model

In [28]:
tuner = kt.Hyperband(model_1D_CNN,
                     objective = 'val_loss', 
                     max_epochs = 10,
                     overwrite=True,
                     factor = 5)

In [29]:
tuner.search(X_train, y_train, epochs = 100, batch_size=12,  validation_data = (X_val, y_val), callbacks=[tf.keras.callbacks.EarlyStopping('val_loss', patience=20)])
# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]


Trial 13 Complete [00h 00m 03s]
val_loss: 0.25727805495262146

Best val_loss So Far: 0.1983036994934082
Total elapsed time: 00h 00m 24s
INFO:tensorflow:Oracle triggered exit


In [30]:
# 배치 정규화 버전
print(f"""
conv_1_filter : {best_hps.get('conv_1')}
conv_1_kernel : {best_hps.get('conv_1_kernel')}


dense_1 : {best_hps.get('dense_1')}
Dropout : {best_hps.get('Dropout')}

dense_2 : {best_hps.get('dense_2')}
alpha : {best_hps.get('alpha')}
""")
# pool_size : {best_hps.get('pool_size')}


conv_1_filter : 80
conv_1_kernel : 5


dense_1 : 112
Dropout : 0.25

dense_2 : 96
alpha : 0.5



# 2D-CNN

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X_2D, Y, test_size=0.2, random_state=42)
print('X_train의 shape: ', X_train.shape ,', X_test의 shape:',X_test.shape,
     ', y_train의 shape: ', y_train.shape ,', y_test의 shape:',y_test.shape)

X_train, X_val, y_train, y_val  = train_test_split(X_train, y_train, test_size=0.1, random_state=42)
print('X_train의 shape: ', X_train.shape ,', X_test의 shape:',X_val.shape,
     ', y_train의 shape: ', y_train.shape ,', y_test의 shape:',y_val.shape)

X_train의 shape:  (395, 5, 8) , X_test의 shape: (99, 5, 8) , y_train의 shape:  (395, 36) , y_test의 shape: (99, 36)
X_train의 shape:  (355, 5, 8) , X_test의 shape: (40, 5, 8) , y_train의 shape:  (355, 36) , y_test의 shape: (40, 36)


In [32]:
from tensorflow.keras.layers import Embedding, Dropout, Flatten, Dense, Conv2D, MaxPooling2D, BatchNormalization, Activation

def model_2D_CNN(hp):
    # create model object
    model = keras.Sequential()
    
    input_shape = (5,8,1)

    model = Sequential()
    model.add(Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [2,3,4]),
        # activation='relu',
        input_shape = input_shape, padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [2,3,4]),
        # activation='relu',
        padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(
        filters=hp.Int('conv_3_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_3_kernel', values = [2,3,4]),
        # activation='relu',
        padding='same'))
    model.add(BatchNormalization())
    '''
    model.add(MaxPooling2D(pool_size=hp.Int('MaxPooling2D', min_value=1, max_value=5, step=1)
                          ))
    '''
    model.add(Activation('relu'))
    
    model.add(Flatten())

    model.add(keras.layers.Dense(
        units=hp.Int('dense_1', min_value=32, max_value=128, step=16),
        activation='relu'
    ))
    model.add(Dropout(hp.Float('Dropout', min_value=0.00, max_value=0.5, step=0.05)))    
        

    model.add(keras.layers.Dense(
        units=hp.Int('dense_2', min_value=32, max_value=128, step=16),
        activation='relu'
    ))
    model.add(Dense(36,
                     activation=keras.layers.LeakyReLU(alpha=hp.Float('alpha', min_value=0.00, max_value=5, step=0.05))
                    ))
    
    #compilation of modelv
    model.compile(loss='mse', optimizer='adam', metrics=['mae'])
    return model

In [33]:
tuner = kt.Hyperband(model_2D_CNN,
                     objective = 'val_loss', 
                     max_epochs = 10,
                     overwrite=True,
                     factor = 5)

In [34]:
tuner.search(X_train, y_train, epochs = 100, batch_size=24,  validation_data = (X_val, y_val), callbacks=[tf.keras.callbacks.EarlyStopping('val_loss', patience=20)])
# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]


Trial 13 Complete [00h 00m 03s]
val_loss: 0.9171992540359497

Best val_loss So Far: 0.8465744256973267
Total elapsed time: 00h 00m 25s
INFO:tensorflow:Oracle triggered exit


In [35]:
print(f"""
conv_1_filter : {best_hps.get('conv_1_filter')}
conv_1_kernel : {best_hps.get('conv_1_kernel')}
conv_2_filter : {best_hps.get('conv_2_filter')}
conv_2_kernel : {best_hps.get('conv_2_kernel')}
conv_3_filter : {best_hps.get('conv_3_filter')}
conv_3_kernel : {best_hps.get('conv_3_kernel')}

dense_1 : {best_hps.get('dense_1')}
Dropout : {best_hps.get('Dropout')}

dense_2 : {best_hps.get('dense_2')}
alpha : {best_hps.get('alpha')}
""")


conv_1_filter : 80
conv_1_kernel : 4
conv_2_filter : 80
conv_2_kernel : 3
conv_3_filter : 64
conv_3_kernel : 3

dense_1 : 96
Dropout : 0.4

dense_2 : 80
alpha : 2.95



In [36]:
def model_2D_CNN(hp):
    # create model object
    model = keras.Sequential()
    
    # adding first convolutional layer 
    model.add(keras.layers.Conv2D(
        filters= hp.Int('conv_1', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(5,8,1)
    ))

    # adding flatten layer    
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(
        units=hp.Int('dense_1', min_value=32, max_value=128, step=16),
        activation= 'relu'
    ))
    # adding dropout layer 
    model.add(Dropout(hp.Float('Dropout', min_value=0.00, max_value=0.5, step=0.05)))
    
    # adding dense layer    
    model.add(keras.layers.Dense(
        units=hp.Int('dense_2', min_value=32, max_value=128, step=16),
        activation= 'relu'
    ))
    # output layer    
    model.add(Dense(36, keras.layers.LeakyReLU(alpha=hp.Float('alpha', min_value=0.00, max_value=0.5, step=0.05))))
    
    #compilation of model
    model.compile(loss='mse', optimizer='adam', metrics=['mae'])
    return model

In [37]:
tuner = kt.Hyperband(model_2D_CNN,
                     objective = 'val_loss', 
                     max_epochs = 10,
                     overwrite=True,
                     factor = 5)

In [38]:
tuner.search(X_train, y_train, epochs = 100, batch_size=24,  validation_data = (X_val, y_val), callbacks=[tf.keras.callbacks.EarlyStopping('val_loss', patience=20)])
# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]


Trial 13 Complete [00h 00m 02s]
val_loss: 0.25150448083877563

Best val_loss So Far: 0.23518891632556915
Total elapsed time: 00h 00m 17s
INFO:tensorflow:Oracle triggered exit


In [39]:
print(f"""
conv_1_filter : {best_hps.get('conv_1')}
conv_1_kernel : {best_hps.get('conv_1_kernel')}

dense_1 : {best_hps.get('dense_1')}
Dropout : {best_hps.get('Dropout')}
dense_2 : {best_hps.get('dense_2')}
alpha : {best_hps.get('alpha')}
""")


conv_1_filter : 48
conv_1_kernel : 3

dense_1 : 48
Dropout : 0.05
dense_2 : 96
alpha : 0.5

